In [7]:
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from FeatureEngineering import create_time_matrix
from MatchingAlgorithm import run_algorithm

In [14]:
orders = pd.read_csv("Simulated Data/order_data_1576.csv")
phleb = pd.read_csv("Simulated Data/phleb_data_1576.csv")
catchment = pd.read_csv("Simulated Data/catchment_data_1576.csv")

orders = orders.iloc[:23]
phleb = phleb.iloc[:3]
catchment = catchment.iloc[:1]

API_key = '' #INPUT YOUR OWN API KEY

In [15]:
result = run_algorithm(orders, catchment, phleb, API_key)

In [17]:
import json

json_object = json.loads(result)
routes = json_object['Routes']
routes = pd.json_normalize(routes)
routes

,Phlebotomist Index,Printable Route,Total Travel Time,Total Loads,Locations Sequence,Start Times Sequence,End Times Sequence,Slack Times Sequence
0,0,Route for Phlebotomist 0:\nLocation 1 Start(42...,109,9,"[1, 26, 9, 16, 10, 15, 21, 13, 4, 7, 0]","[[420, 420], [430, 454], [478, 502], [539, 551...","[[420, 420], [471, 495], [522, 546], [565, 577...","[[0, 24], [0, 24], [0, 24], [0, 12], [2, 37], ..."
1,1,Route for Phlebotomist 1:\nLocation 2 Start(42...,68,6,"[2, 23, 24, 11, 12, 17, 20, 0]","[[420, 420], [430, 440], [463, 473], [523, 533...","[[420, 420], [456, 466], [507, 517], [563, 573...","[[0, 10], [0, 10], [0, 10], [0, 10], [13, 23],..."
2,2,Route for Phlebotomist 2:\nLocation 3 Start(42...,81,7,"[3, 19, 6, 14, 5, 8, 18, 0]","[[420, 420], [448, 457], [515, 524], [554, 563...","[[420, 420], [505, 514], [541, 550], [581, 590...","[[15, 24], [0, 9], [0, 9], [0, 9], [192, 201],..."


In [18]:
import json

def get_vacant_time_windows(matching_algo_results):
    json_object = json.loads(matching_algo_results)
    data = []
    routes = json_object['Routes']

    for phleb in range(len(routes)):
        curr_phleb = routes[phleb]
        phleb_index = curr_phleb['Phlebotomist Index']
        slack_times = curr_phleb['Slack Times Sequence']
        locations = curr_phleb['Locations Sequence']
        start_times = curr_phleb['Start Times Sequence']
        end_times = curr_phleb['End Times Sequence']
        for index in range(len(slack_times)):
            slack_time = slack_times[index]
            slack_time_min = slack_time[0]
            slack_time_max = slack_time[1]
            if slack_time_max == 0: #no slack time between this loc and the next
                continue
            else:
                # Case 1: Phlebotomist ends his/her service at the earliest possible time. Slack time before next loc should be max
                # Phlebotomist immediately becomes available at his/her current location
                temp_data = []
                temp_data.append(phleb_index)
                temp_data.append(end_times[index][0]) # Time window start  - take earliest possible time
                temp_data.append(end_times[index][0] + slack_time_max) # Time window end. Adding the max slack time
                temp_data.append(slack_time_max)
                temp_data.append(locations[index]) # Last location before free
                temp_data.append(locations[index + 1]) # First location after free
                data.append(temp_data)

                # Case 2: Phlebotomist ends his/her service at the latest possible time. Slack time before next loc should be min
                # Phlebotomist immediately becomes available at his/her current location
                ### CASE 2 OMITTED DUE TO IT BEING SUBSET OF CASE 1 (can uncomment this chunk to visualize if needed) ###
                '''
                if index != 0:
                    temp_data = []
                    temp_data.append(phleb_index)
                    temp_data.append(end_times[index][1]) # Time window start - take latest possible time
                    temp_data.append(end_times[index][1] + slack_time_min) # Time window end. Adding the min slack time
                    temp_data.append(slack_time_min)
                    temp_data.append(locations[index]) # Last location before free
                    temp_data.append(locations[index + 1]) # First location after free
                    data.append(temp_data)
                '''

                # Case 3: Phlebotomist ends his/her service at the earliest possible time. Slack time before next loc should be max
                # Phlebotomist immediately travels to his/her next location, and becomes available there
                temp_data = []
                temp_data.append(phleb_index)
                travel_time = start_times[index + 1][0] - end_times[index][1] - slack_time_min
                temp_data.append(end_times[index][0] + travel_time) # Time window start - take earliest possible time + travel time to next loc
                temp_data.append(start_times[index + 1][1]) # Time window end, which is basically the latest possible start time of the next loc (i.e. the loc that the phlebo has traveled to)
                temp_data.append(start_times[index + 1][1] - end_times[index][0] - travel_time)
                temp_data.append(locations[index + 1]) # Last location before free
                temp_data.append(locations[index + 1]) # First location after free
                data.append(temp_data)

                # Case 4: Phlebotomist ends his/her service at the latest possible time. Slack time before next loc should be min
                # Phlebotomist immediately travels to his/her next location, and becomes available there
                ### CASE 4 OMITTED DUE TO IT BEING SUBSET OF CASE 3 (can uncomment this chunk to visualize if needed) ###
                '''
                if index != 0:
                    temp_data = []
                    temp_data.append(phleb_index)
                    travel_time = start_times[index + 1][0] - end_times[index][1] - slack_time_min
                    temp_data.append(end_times[index][1] + travel_time) # Time window start - take latest possible time + travel time to next loc
                    temp_data.append(start_times[index + 1][1]) # Time window end, which is basically the latest possible start time of the next loc (i.e. the loc that the phlebo has traveled to)
                    temp_data.append('null')
                    temp_data.append(locations[index + 1]) # Last location before free
                    temp_data.append(locations[index + 1]) # First location after free
                    data.append(temp_data)
                '''

    vacant_time_windows = pd.DataFrame(columns=['PhlebotomistIndex', 'TimeWindowStart', 'TimeWindowEnd', 'TimeWindowDuration',
                                                'LastLocBeforeFree', 'FirstLocAfterFree'],
                                      data=data)

    return vacant_time_windows

In [19]:
get_vacant_time_windows(result)

,PhlebotomistIndex,TimeWindowStart,TimeWindowEnd,TimeWindowDuration,LastLocBeforeFree,FirstLocAfterFree
0,0,420,444,24,1,26
1,0,430,454,24,26,26
2,0,471,495,24,26,9
3,0,454,502,48,9,9
4,0,522,546,24,9,16
5,0,515,551,36,16,16
6,0,565,577,12,16,10
7,0,564,588,24,10,10
8,0,632,669,37,10,15
9,0,648,685,37,15,15
